# InsightPlaces

A imobiliária `InsightPlaces`, situada na cidade do Rio de Janeiro, está enfrentando dificuldades para alugar e vender imóveis. Em uma pesquisa de como empresas semelhantes operam no mercado, a `InsightPlaces` percebeu que esse problema pode estar relacionado aos valores dos imóveis e às recomendações que faz.

Perguntas a serem respondidas:
* Como podemos definir de forma eficiente os preços dos imóveis lidando com grande volume de dados?
* É importante recomendar imóveis utilizando outro critério?
* O que precisa ser feito?

## Spark no Colab
---

In [52]:
# instalar as dependências
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
!tar xf spark-3.3.1-bin-hadoop3.tgz
!pip install -q findspark

In [53]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

In [54]:
import findspark
findspark.init()

## Carregamento de Spark
---

In [55]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

In [56]:
spark

### Carregando os dados das InsightPlaces


In [57]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:
'''# baixando o aquivo
from urllib import request

file_url = 'https://caelum-online-public.s3.amazonaws.com/challenge-spark/semana-1.zip'
file = '/content/drive/MyDrive/InsightPlaces/dados/semana-1.zip'
request.urlretrieve(file_url , file )'''

"# baixando o aquivo\nfrom urllib import request\n\nfile_url = 'https://caelum-online-public.s3.amazonaws.com/challenge-spark/semana-1.zip'\nfile = '/content/drive/MyDrive/InsightPlaces/dados/semana-1.zip'\nrequest.urlretrieve(file_url , file )"

In [59]:
'''import zipfile

zipfile.ZipFile('/content/drive/MyDrive/InsightPlaces/dados/semana-1.zip','r').extractall('/content/drive/MyDrive/InsightPlaces/dados/')'''

"import zipfile\n\nzipfile.ZipFile('/content/drive/MyDrive/InsightPlaces/dados/semana-1.zip','r').extractall('/content/drive/MyDrive/InsightPlaces/dados/')"

## Análises Preliminares

In [60]:
path = '/content/drive/MyDrive/InsightPlaces/dados/dataset_bruto.json'
df = spark.read.json(path)

In [61]:
df.limit(5).show()

+--------------------+--------------------+--------------------+
|             anuncio|             imagens|             usuario|
+--------------------+--------------------+--------------------+
|{0, [], [16], [0]...|[{39d6282a-71f3-4...|{9d44563d-3405-4e...|
|{0, [], [14], [0]...|[{23d2b3ab-45b0-4...|{36245be7-70fe-40...|
|{0, [1026], [1026...|[{1da65baa-368b-4...|{9dc415d8-1397-4d...|
|{0, [120], [120],...|[{79b542c6-49b4-4...|{9911a2df-f299-4a...|
|{0, [3], [3], [0]...|[{e2bc497b-6510-4...|{240a7aab-12e5-40...|
+--------------------+--------------------+--------------------+



In [62]:
n_linhas = df.count()
n_colunas = len(df.columns)

print(f'No Dataset possuimos {n_linhas} linhas e {n_colunas} colunas.')

No Dataset possuimos 89083 linhas e 3 colunas.


In [63]:
print('A estrutura das colunas do dataset: \n')
display(df.printSchema())

A estrutura das colunas do dataset: 

root
 |-- anuncio: struct (nullable = true)
 |    |-- andar: long (nullable = true)
 |    |-- area_total: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- area_util: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- banheiros: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- caracteristicas: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- endereco: struct (nullable = true)
 |    |    |-- bairro: string (nullable = true)
 |    |    |-- cep: string (nullable = true)
 |    |    |-- cidade: string (nullable = true)
 |    |    |-- estado: string (nullable = true)
 |    |    |-- latitude: double (nullable = true)
 |    |    |-- longitude: double (nullable = true)
 |    |    |-- pais: string (nullable = true)
 |    |    |-- rua: string (nullable = true)
 |    |    |-- zona: string (nullable = true)
 |    |--

None

## Separando dados das colunas `anuncio`, `imagens` e `usuario`

In [230]:
anuncio = df.select('anuncio.*')

anuncio.limit(5).show(truncate= False)

+-----+----------+---------+---------+---------------------------------------+-----------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+----------------------------+
|andar|area_total|area_util|banheiros|caracteristicas                        |endereco                                                                                                                     |id                                  |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|valores                     |
+-----+----------+---------+---------+---------------------------------------+-----------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+----------------------------+
|0    |[]    

In [231]:
imagens = df.select(['imagens.id', 'imagens.url'])

imagens.limit(5).show(truncate= False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id                                                                                                                                                      |url                                                                                                                                                                                                                                                                       

In [232]:
usuario = df.select(['usuario.id', 'usuario.nome'])

usuario.limit(5).show(truncate= False)

+------------------------------------+--------+
|id                                  |nome    |
+------------------------------------+--------+
|9d44563d-3405-4e84-9381-35b7cf40a9a4|Frank   |
|36245be7-70fe-40cd-84d4-70a57dc8f976|Caroline|
|9dc415d8-1397-4d8d-8425-b8310fad309d|Oliver  |
|9911a2df-f299-4a73-a384-281417abaf69|Matthew |
|240a7aab-12e5-4069-9a2c-9dca4c4f9c68|Jude    |
+------------------------------------+--------+



## Filtrar a base de dados

In [233]:
print('Contagem dos dados para anáçose das colunas "tipo_uso", "tipo_unidade", "tipo_anuncio"')
anuncio.groupby(['tipo_uso', 'tipo_unidade', 'tipo_anuncio']).count().orderBy('count', ascending= False).show()

Contagem dos dados para anáçose das colunas "tipo_uso", "tipo_unidade", "tipo_anuncio"
+-----------+------------+------------+-----+
|   tipo_uso|tipo_unidade|tipo_anuncio|count|
+-----------+------------+------------+-----+
|Residencial| Apartamento|       Usado|66562|
|Residencial|        Casa|       Usado|10224|
|Residencial|      Outros|       Usado| 7502|
|  Comercial|      Outros|       Usado| 4443|
|Residencial| Apartamento|  Lançamento|  235|
|  Comercial|        Casa|       Usado|   92|
|Residencial|      Outros|  Lançamento|   15|
|  Comercial| Apartamento|       Usado|    4|
|  Comercial|      Outros|  Lançamento|    3|
|Residencial|        Casa|  Lançamento|    3|
+-----------+------------+------------+-----+



### Separando DataFrame `tipo_uso`

In [234]:
print('Valores distintos da coluna tipo_uso:')
anuncio.select('tipo_uso').distinct().show()

Valores distintos da coluna tipo_uso:
+-----------+
|   tipo_uso|
+-----------+
|  Comercial|
|Residencial|
+-----------+



In [235]:
n_linhas_tipo_uso = anuncio.where('tipo_uso == "Residencial"').count()
print(f'Temos {n_linhas_tipo_uso} imoveis "Residencial" na coluna "tipo_uso"')

Temos 84541 imoveis "Residencial" na coluna "tipo_uso"


In [236]:
print('Primeiro filtro "tipo_uso == Residencial":')
anuncio.filter('tipo_uso == "Residencial"').limit(5).show()

Primeiro filtro "tipo_uso == Residencial":
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    0|     [120]|    [120]|      [0]|[Portão eletrônic...|{Campo Grande, 23...|ac6cb348-69d6-45a...|    [0]|    []|       Usado|      Outros|Residencial|  []|[{80, null, Venda...|
|    0|       [3]|      [3]|      [0]|                  []|{São Cristóvão, 2...|e032b908-ef42-4d4...|    [0]|    []|       Usado|      Outros|Residencial|  []|[{0, 0, Venda, 50...|
|    3|      [43]|     [43]|      [1]|[Academia, Chu

### Separando DataFrame `tipo_unidade`

In [237]:
print('Valores distintos da coluna tipo_unidade:')
anuncio.select('tipo_unidade').distinct().show()

Valores distintos da coluna tipo_unidade:
+------------+
|tipo_unidade|
+------------+
|      Outros|
| Apartamento|
|        Casa|
+------------+



In [238]:
n_linhas_tipo_unidade = anuncio.where('tipo_unidade == "Apartamento"').count()
print(f'Temos {n_linhas_tipo_unidade} imoveis "Apartamento" na coluna "tipo_unidade"')

Temos 66801 imoveis "Apartamento" na coluna "tipo_unidade"


In [239]:
print('Primeiro filtro "tipo_unidade == Apartamento":')
anuncio.filter('tipo_unidade == "Apartamento"').limit(5).show()

Primeiro filtro "tipo_unidade == Apartamento":
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    3|      [43]|     [43]|      [1]|[Academia, Churra...|{Paciência, 23585...|d2e3a3aa-09b5-45a...|    [2]|    []|       Usado| Apartamento|Residencial| [1]|[{245, null, Vend...|
|    2|      [42]|     [42]|      [1]|[Churrasqueira, P...|{Paciência, 23585...|085bab2c-87ad-452...|    [2]|    []|       Usado| Apartamento|Residencial| [1]|[{0, 0, Venda, 15...|
|    1|      [41]|     [41]|      [1]|[Portaria 

### Separando DataFrame `tipo_anuncio`

In [240]:
print('Valores distintos da coluna tipo_anuncio:')
anuncio.select('tipo_anuncio').distinct().show()

Valores distintos da coluna tipo_anuncio:
+------------+
|tipo_anuncio|
+------------+
|       Usado|
|  Lançamento|
+------------+



In [241]:
n_linhas_tipo_anuncio = anuncio.where('tipo_anuncio == "Usado"').count()
print(f'Temos {n_linhas_tipo_anuncio} imoveis "Usado" na coluna "tipo_anuncio"')

Temos 88827 imoveis "Usado" na coluna "tipo_anuncio"


In [242]:
print('Primeiro filtro "tipo_anuncio == Usado":')
anuncio.filter('tipo_anuncio == "Usado"').limit(5).show()

Primeiro filtro "tipo_anuncio == Usado":
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    0|        []|     [16]|      [0]|                  []|{Centro, 20061003...|47d553e0-79f2-4a4...|    [0]|   [0]|       Usado|      Outros|  Comercial| [1]|[{260, 107, Venda...|
|    0|        []|     [14]|      [0]|                  []|{Centro, 20051040...|b6ffbae1-17f6-487...|    [0]|    []|       Usado|      Outros|  Comercial| [0]|[{260, 107, Venda...|
|    0|    [1026]|   [1026]|      [0]|                

### Realizando filtro composto

Foi solicitado que fizéssemos alguns filtros nas colunas tipo_uso, tipo_unidade e tipo_anuncio da nossa base de dados:

* tipo_uso: Residencial;
* tipo_unidade: Apartamento;
* tipo_anuncio: Usado.

In [243]:
n_linhas_filtro = anuncio.where('tipo_uso == "Residencial" and tipo_unidade == "Apartamento" and tipo_anuncio == "Usado"').count()
print(f'Temos {n_linhas_filtro} imoveis de acordo com o filtro')

Temos 66562 imoveis de acordo com o filtro


In [244]:
anuncio.filter('tipo_uso == "Residencial" and tipo_unidade == "Apartamento" and tipo_anuncio == "Usado"').limit(5).show()

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    3|      [43]|     [43]|      [1]|[Academia, Churra...|{Paciência, 23585...|d2e3a3aa-09b5-45a...|    [2]|    []|       Usado| Apartamento|Residencial| [1]|[{245, null, Vend...|
|    2|      [42]|     [42]|      [1]|[Churrasqueira, P...|{Paciência, 23585...|085bab2c-87ad-452...|    [2]|    []|       Usado| Apartamento|Residencial| [1]|[{0, 0, Venda, 15...|
|    1|      [41]|     [41]|      [1]|[Portaria 24h, Co...|{Guaratiba, 23036...|18d22cbe-1b86-4

## Transformar os Data Types das colunas

Será transformado os dados das colunas `quartos`, `suites`, `banheiros`, `vaga`, `area_total` e `area_util` de listas para inteiros.

In [245]:
from pyspark.sql.types import IntegerType
from pyspark.sql import functions as f

In [246]:
anuncio.select(['quartos', 'suites', 'banheiros', 'vaga', 'area_total', 'area_util']).printSchema()

root
 |-- quartos: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- suites: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- banheiros: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- vaga: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- area_total: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- area_util: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [247]:
anuncio = anuncio\
    .withColumn('quartos', anuncio['quartos'][0].cast(IntegerType())) \
    .withColumn('suites', anuncio['suites'][0].cast(IntegerType())) \
    .withColumn('banheiros', anuncio['banheiros'][0].cast(IntegerType())) \
    .withColumn('vaga', anuncio['vaga'][0].cast(IntegerType())) \
    .withColumn('area_total', anuncio['area_total'][0].cast(IntegerType())) \
    .withColumn('area_util', anuncio['area_util'][0].cast(IntegerType())) \

In [248]:
anuncio.select(['quartos', 'suites', 'banheiros', 'vaga', 'area_total', 'area_util']).printSchema()

root
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- vaga: integer (nullable = true)
 |-- area_total: integer (nullable = true)
 |-- area_util: integer (nullable = true)



In [249]:
anuncio.limit(5).show()

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    0|      null|       16|        0|                  []|{Centro, 20061003...|47d553e0-79f2-4a4...|      0|     0|       Usado|      Outros|  Comercial|   1|[{260, 107, Venda...|
|    0|      null|       14|        0|                  []|{Centro, 20051040...|b6ffbae1-17f6-487...|      0|  null|       Usado|      Outros|  Comercial|   0|[{260, 107, Venda...|
|    0|      1026|     1026|        0|                  []|{Maria da Graça, ...|1fb030a5-9e3e-4

## Tratamento de informações sobre localização

Ao analisarmos as colunas de dentro de `endereco`, foi solicitado que apenas as informações sobre `bairro` e `zona` da cidade fossem extraídas.


In [250]:
print('Estrutura da colunas endereco: ')
endereco = anuncio.select('endereco')
endereco.printSchema()

Estrutura da colunas endereco: 
root
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)



Então, será analisado a coluna endereco e utilizadas apenas as informações sobre `bairro` e `zona` em colunas de nosso DataFrame.

In [251]:
# salvando infomações para possibilidades futuras
endereco = endereco \
    .withColumn('bairro', endereco['endereco']['bairro']) \
    .withColumn('zona', endereco['endereco']['zona'])

endereco.limit(5).show()

+--------------------+--------------+------------+
|            endereco|        bairro|        zona|
+--------------------+--------------+------------+
|{Centro, 20061003...|        Centro|Zona Central|
|{Centro, 20051040...|        Centro|Zona Central|
|{Maria da Graça, ...|Maria da Graça|  Zona Norte|
|{Campo Grande, 23...|  Campo Grande|  Zona Oeste|
|{São Cristóvão, 2...| São Cristóvão|  Zona Norte|
+--------------------+--------------+------------+



In [252]:
anuncio = anuncio \
    .withColumn('bairro', endereco['endereco']['bairro']) \
    .withColumn('zona', endereco['endereco']['zona'])

anuncio.limit(5).show()


+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------+------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|        bairro|        zona|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------+------------+
|    0|      null|       16|        0|                  []|{Centro, 20061003...|47d553e0-79f2-4a4...|      0|     0|       Usado|      Outros|  Comercial|   1|[{260, 107, Venda...|        Centro|Zona Central|
|    0|      null|       14|        0|                  []|{Centro, 20051040...|b6ffbae1-17f6-487...|      0|  null|       Usado|      Outros|  Comercial|   0|[{260

## Transformar coluna `valores`

Pensando em simplificar a compreensão dos dados para as pessoas cientistas de dados, vamos entender a estrutura da coluna valores que é a mais importante da base de dados.



In [253]:
anuncio.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: integer (nullable = true)
 |-- area_util: integer (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: integer (nullable = true)
 |-- valores: array (nullable = true)
 

In [254]:
valores = anuncio.select('valores')

valores.printSchema()

root
 |-- valores: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- condominio: string (nullable = true)
 |    |    |-- iptu: string (nullable = true)
 |    |    |-- tipo: string (nullable = true)
 |    |    |-- valor: string (nullable = true)



Podemos observar que os valores são do tipo array, ou seja, uma lista que contém elementos. Estes elementos são dicionários com as informações de `condominio`, `iptu`, `tipo`, e `valor`

Nesse contexto, transforme os dados da coluna valores em colunas separadas.

In [255]:
# salvando infomações para possibilidades futuras
valores = valores \
    .withColumn('condominio', anuncio['valores']['condominio'][0]) \
    .withColumn('iptu', anuncio['valores']['iptu'][0]) \
    .withColumn('tipo', anuncio['valores']['tipo'][0]) \
    .withColumn('valor', anuncio['valores']['valor'][0])

valores.limit(5).show()

+--------------------+----------+----+-----+-----+
|             valores|condominio|iptu| tipo|valor|
+--------------------+----------+----+-----+-----+
|[{260, 107, Venda...|       260| 107|Venda|10000|
|[{260, 107, Venda...|       260| 107|Venda|10000|
|[{null, 1613, Ven...|      null|1613|Venda|10000|
|[{80, null, Venda...|        80|null|Venda|10000|
|[{0, 0, Venda, 50...|         0|   0|Venda| 5000|
+--------------------+----------+----+-----+-----+



In [256]:
anuncio = anuncio \
    .withColumn('condominio', anuncio['valores']['condominio'][0]) \
    .withColumn('iptu', anuncio['valores']['iptu'][0]) \
    .withColumn('tipo', anuncio['valores']['tipo'][0]) \
    .withColumn('valor', anuncio['valores']['valor'][0])

anuncio.limit(5).show()

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------+------------+----------+----+-----+-----+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|        bairro|        zona|condominio|iptu| tipo|valor|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------+------------+----------+----+-----+-----+
|    0|      null|       16|        0|                  []|{Centro, 20061003...|47d553e0-79f2-4a4...|      0|     0|       Usado|      Outros|  Comercial|   1|[{260, 107, Venda...|        Centro|Zona Central|       260| 107|Venda|10000|
|    0|      null|       14|        0|              

## Realizando filtro na coluna `valores.tipo`

In [257]:
print('Valores distintos na coluna "tipo":')
anuncio.select('tipo').distinct().show()

Valores distintos na coluna "tipo":
+-------+
|   tipo|
+-------+
|Aluguel|
|  Venda|
+-------+



In [258]:
n_linhas_venda = anuncio.filter('tipo == "Venda"').count()
print(f'O dataset possui {n_linhas_venda} imoveis a venda')

O dataset possui 88272 imoveis a venda


In [259]:
print('Visualizando as linhas "Venda" na coluna "tipo":')
anuncio.filter('tipo == "Venda"').limit(5).show()

Visualizando as linhas "Venda" na coluna "tipo":
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------+------------+----------+----+-----+-----+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|        bairro|        zona|condominio|iptu| tipo|valor|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------+------------+----------+----+-----+-----+
|    0|      null|       16|        0|                  []|{Centro, 20061003...|47d553e0-79f2-4a4...|      0|     0|       Usado|      Outros|  Comercial|   1|[{260, 107, Venda...|        Centro|Zona Central|       260| 107|Venda|10000|
|  

## Salvando o arquivo

In [260]:
%%time
anuncio.write.parquet(
    path= '/content/drive/MyDrive/InsightPlaces/dados/anuncio_tratado_parquet',
    mode= 'overwrite')

CPU times: user 33.4 ms, sys: 9.55 ms, total: 42.9 ms
Wall time: 4.3 s


### Salvando formato CSV

* Obs: para salvar como arquivo CSV é necessario moodificarmos o Type da coluna `caracteristicas` e realizar um drop nas colunas `endereco` e `valores`

In [261]:
from pyspark.sql.types import StringType

In [262]:
anuncio.select('caracteristicas').printSchema()

root
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [264]:
anuncio = anuncio.withColumn('caracteristicas', anuncio['caracteristicas'].cast(StringType()))
anuncio.select('caracteristicas').printSchema()

root
 |-- caracteristicas: string (nullable = true)



In [265]:
anuncio = anuncio.drop('endereco')
anuncio = anuncio.drop('valores')

anuncio.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: integer (nullable = true)
 |-- area_util: integer (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- caracteristicas: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: integer (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: string (nullable = true)
 |-- iptu: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: string (nullable = true)



In [266]:
%%time
anuncio.write.format("csv").save('/content/drive/MyDrive/InsightPlaces/dados/anuncio_tratado_csv')

CPU times: user 36.9 ms, sys: 3.43 ms, total: 40.3 ms
Wall time: 4.69 s
